# A Perfect Reconstruction Filter Bank (PRFB)
Using MST filters.

```
             La      L       Lb           Lc
   +--> K_0 ---> d2 ---> u2 ---> 1/2*K_0 ---+
   |                                        |
x  |                                        | _x
---+                                        +--->
   |                                        |
   |                                        |
   +--> K_1 ---> d2 ---> u2 ---> 1/2*K_1 ---+
             Ha      H       Hb           Hc
```

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sounddevice as sd

In [ ]:
def plot(x, y, xlabel='', ylabel='', title=''):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.grid()
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_text(ylabel)
    ax.plot(x, y, '.', markersize=1)
    plt.show(block=False)

In [ ]:
fs = 44100
duration = 5.0  # seconds
x = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype=np.int16)
print("Say something!")
while sd.wait():
    pass
print("done")

In [ ]:
sd.play(x)
plot(
    np.linspace(0, len(x)-1, len(x)), x, "Time", "Amplitude", "Audio Signal")

### Low-frequency subband (analysis)
We convolve $x$ with $K_0=[1, 1]$.

In [ ]:
La = np.convolve(x[:, 0], [1.0, 1.0])

In [ ]:
for i in range(20):
    print(float(x[i][0]), end= ' ')
print('*'*10)
for i in range(20):
    print(La[i], end= ' ')

In [ ]:
sd.play((La//2).astype(np.int16))
plot(np.linspace(0, len(La)-1, len(La)), La, "Time", "Amplitude", "Low-frequency Subband (without subsampling)")

Notice that the gain of this filter is 2.

### High-frequency subband (analysis)
We convolve $x$ with $K_1=[1, -1]$.

In [ ]:
Ha = np.convolve(x[:, 0], [1.0, -1.0])

In [ ]:
for i in range(20):
    print(float(x[i][0]), end= ' ')
print('*'*10)
for i in range(20):
    print(Ha[i], end= ' ')

In [ ]:
sd.play((Ha//2).astype(np.int16))
plot(np.linspace(0, len(Ha)-1, len(Ha)), Ha, "Time", "Amplitude", "High-frequency Subband (without subsampling)")

Now it's more difficult to see that the signal gain of $K_1$ is 2, but this is the real gain.

### Subsampling (decimation)
Set to zero half of the transform coefficients.

In [ ]:
Lb = La.copy()
Hb = Ha.copy()
Lb[1::2] = 0
Hb[1::2] = 0

### Low-frequency subband (synthesis)

In [ ]:
Lc = np.convolve(Lb, [0.5, 0.5])

In [ ]:
for i in range(20):
    print(float(Lb[i]), end= ' ')
print()
for i in range(20):
    print(Lc[i], end= ' ')

### High frequency subband (synthesis)

In [ ]:
Hc = np.convolve(Hb, [0.5, -0.5])

In [ ]:
for i in range(20):
    print(float(Hb[i]), end= ' ')
print()
for i in range(20):
    print(Hc[i], end= ' ')

### Add the subbans

In [ ]:
_x = Lc + Hc

In [ ]:
_x_ = np.empty_like(_x)
_x_[0::2] = _x[0::2]
_x_[1::2] = np.roll(_x[1::2],-1) # The even samples are delayed

In [ ]:
_x_ = _x_.astype(np.int16).reshape((len(_x),1))

In [ ]:
sd.play(_x_)
plot(np.linspace(0, len(_x_)-1, len(_x_)), _x_, "Time", "Amplitude", "Audio Signal")

### Perfect reconstruction test

In [ ]:
(x == _x_[:-2]).all()

In [ ]:
for i in range(20):
    print(x[i][0], end= ' ')

In [ ]:
for i in range(20):
    print(_x_[i][0], end= ' ')